[View in Colaboratory](https://colab.research.google.com/github/nautilusPrime/cifar10-image-classifier/blob/master/cifar10.ipynb)

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 100
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170508288/170498071 [==============================] - 23s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# You can modify Model here.
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [5]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [6]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [7]:
x_train = x_train.astype('float16')
x_test = x_test.astype('float16')
x_train /= 255
x_test /= 255

In [9]:
data_augmentation = False

In [10]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), samples_per_epoch=len(x_train),
                        epochs=epochs, validation_data=(x_test, y_test), workers=4)

Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/100
18144/50000 [=========>....................] - ETA: 31s - loss: 2.2600 - acc: 0.2816

50000/50000 [==============================] - 49s 984us/step - loss: 1.8795 - acc: 0.3627 - val_loss: 1.4051 - val_acc: 0.5056
Epoch 2/100
16544/50000 [========>.....................] - ETA: 29s - loss: 1.4944 - acc: 0.4659

50000/50000 [==============================] - 47s 942us/step - loss: 1.4447 - acc: 0.4892 - val_loss: 1.2232 - val_acc: 0.5690
Epoch 3/100
15968/50000 [========>.....................] - ETA: 30s - loss: 1.3365 - acc: 0.5301

50000/50000 [==============================] - 47s 937us/step - loss: 1.2994 - acc: 0.5462 - val_loss: 1.0819 - val_acc: 0.6193
Epoch 4/100
15776/50000 [========>.....................] - ETA: 30s - loss: 1.2118 - acc: 0.5809

50000/50000 [==============================] - 47s 938us/step - loss: 1.1944 - acc: 0.5857 - val_loss: 1.0393 - val_acc: 0.6294
Epoch 5/100
15712/50000 [========>.....................] - ETA: 30s - loss: 1.1197 - acc: 0.6099

...

50000/50000 [==============================] - 47s 933us/step - loss: 0.2080 - acc: 0.9286 - val_loss: 0.6488 - val_acc: 0.8205
Epoch 97/100
15520/50000 [========>.....................] - ETA: 30s - loss: 0.1954 - acc: 0.9334

50000/50000 [==============================] - 47s 942us/step - loss: 0.2074 - acc: 0.9294 - val_loss: 0.6374 - val_acc: 0.8162
Epoch 98/100
15520/50000 [========>.....................] - ETA: 31s - loss: 0.1956 - acc: 0.9314

50000/50000 [==============================] - 47s 949us/step - loss: 0.2050 - acc: 0.9298 - val_loss: 0.6621 - val_acc: 0.8186
Epoch 99/100
15520/50000 [========>.....................] - ETA: 30s - loss: 0.1936 - acc: 0.9312

50000/50000 [==============================] - 47s 944us/step - loss: 0.2043 - acc: 0.9296 - val_loss: 0.6621 - val_acc: 0.8168
Epoch 100/100
15456/50000 [========>.....................] - ETA: 30s - loss: 0.2069 - acc: 0.9303

50000/50000 [==============================] - 48s 951us/step - loss: 0.2050 - acc: 0.9297 - val_loss: 0.6679 - val_acc: 0.8141


In [0]:
from google.colab import files 

In [0]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/saved_models/keras_cifar10_trained_model.h5 


In [0]:
files.download(model_path)

In [11]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 3s 284us/step
Test loss: 0.6678660587310791
Test accuracy: 0.8141
